In [1]:
using JuMP
using HiGHS
using Ipopt
using Juniper
using DelimitedFiles
using Serialization
using Random

include("utils.jl") 
include("optimize_list.jl")

optimize_list (generic function with 1 method)

In [ ]:
N = 10           # number of examples
horiz = 20       # total horizon

limit_count = 10 # max number of trials to create a new replacement demand in case of infeasibility

createDemands = false

DM_l = [15]
AM_l = [1]

XM_l = [6]            # max queue length 
YM_l = [10]        # max buffer length before dropping calls

serM_nd_l = [3]        # max number of servers original model


# DM_l = [10, 15, 20, 25]
# AM_l = [1, 2]

# XM_l = [6, 9, 12, 15]            # max queue length 
# YM_l = [10, 15, 20, 25]        # max buffer length before dropping calls

# serM_nd_l = [5, 10, 15, 20, 25]        # max number of servers original model
# serM_nd_l = [5, 10, 15, 20, 25]        


compensateSerDelay = false
compensateCostDelay = false

for AM in AM_l
    for DM in DM_l
        for YM in YM_l
            for XM in XM_l
                for serM_nd in serM_nd_l
                    if (AM>=2 && serM_nd>=15)
                        # skip infeasible configurations
                        println("Skipping AM=", AM, " serM_nd=", serM_nd)
                    else
                        global d_prop = distribution_properties(DM, 0, "uniform"); # properties of the demand distribution
                        global a_prop = distribution_properties(AM, 0, "uniform");

                        if createDemands
                            global d_mat = demand_generator_mat(N, horiz, d_prop.M, d_prop.type, d_prop.std_dev);
                            global a_mat = demand_generator_mat(N, horiz, a_prop.M, a_prop.type, a_prop.std_dev);
                        else
                            d_fn = string("exogenous_vars//d_mat_", d_prop.type, "_max_", d_prop.M, "_dev_", d_prop.std_dev, ".txt");
                            a_fn = string("exogenous_vars//a_mat_", a_prop.type, "_max_", a_prop.M, "_dev_", a_prop.std_dev, ".txt");
                            global d_mat = DelimitedFiles.readdlm(d_fn);
                            global a_mat = DelimitedFiles.readdlm(a_fn);
                        end

                        # bounds
                        phiM = 9          # max adimission to queue
                        tserM = 3         # max delay in service time

                        # iniital conditions
                        X0 = 4
                        Y0 = 6
                        L0 = 0
                        Z0 = 1
                        tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

                        # Non-linear objective function costs
                        c_blr_nd = 400
                        c_ser_nd = 1

                        # Linear objective function costs
                        if compensateCostDelay
                            c_Z_d = 3/2/tm      # weight of served clients      
                            c_ser_d = 1/tm      # weight of server cost
                            c_L_d = 1/2/tm      # weight of buffer cost
                        else
                            c_Z_d = 3/2               
                            c_ser_d = 1         
                            c_L_d = 1/2         
                        end

                        # Adjust max number of servers for the model with delay
                        if compensateSerDelay
                            serM_d = Int(serM_nd*tm)                  
                        else
                            serM_d = serM_nd                        
                        end

                        global bds_nd = bounds(XM, YM, phiM, serM_nd, 0)
                        global bds_d = bounds(XM, YM, phiM, serM_d, tserM)

                            
                        global ic = initial_conditions(X0, Y0, L0, Z0)

                        global c_nd = c(0, 0, c_ser_nd, c_blr_nd)  # original model costs
                        global c_d = c(c_Z_d, c_L_d, c_ser_d, c_blr_nd)  # model with delay costs

                        global df_input = df_input_generator(horiz, bds_d.serM, bds_d.tserM);

                        println("1")
                        res_os_d, d_mat_os_d, a_mat_os_d, count_os_d = optimize_list("nd_os", d_mat, a_mat);
                        println("2")
                        res_om_d, d_mat_om_d, a_mat_om_d, count_om_d = optimize_list("nd_om", d_mat, a_mat);
                        println("3")

                        res_list = [res_os_d, res_om_d]

                        a_mat_list = [a_mat_os_d, a_mat_om_d]
                        d_mat_list = [d_mat_os_d, d_mat_om_d]
                        count_list = [count_os_d, count_om_d]

                        for i in 1:length(res_list)
                            res = res_list[i]
                            res_stats = Dict()

                            result_id = res.id*"_"*string(horiz)*"_"*string(d_prop.M)*"_"*string(a_prop.M)*"_"*string(res.bds.XM)*"_"*string(res.bds.YM)*"_"*string(res.bds.phiM)*"_"*string(res.bds.serM)*"_"*string(res.bds.tserM)
                            cost, av_blr, av_dr, av_ser, total_clients = compute_metrics(res, horiz, N)

                            # Print results table
                            println(res.id)
                            data = hcat(av_blr, total_clients, av_dr, av_ser, cost)
                            header = (["average blr", "Average served clients", "Average dropped clients", "Average active Servers", "Average cost function value"]);
                            printTable(data, header)

                            # add csv line
                            add_line_to_csv(result_id, res, horiz, d_prop, a_prop, cost, av_blr, av_dr, av_ser, total_clients, count_list[i])

                            # Save results to file
                            var_dict = Dict("res"=>res, "a"=>a_mat_list[i], "d"=>d_mat_list[i])
                            fn = string("results//",result_id,".txt")
                            serialize(fn, var_dict)
                        end
                    end
                end
            end
        end
    end
end

1

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Bool[